In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy ADK Agent in AI Engine

This notebook provides a step-by-step to deploy an Agent Created using Agent Development Kit on Agent Engine (ReasonEngine on Vertex AI)

**Important**: This notebook consider that the Agent was built with ADK and the agent files are inside an agent folder and the dependencies in a file config.yaml

Folders structure (example): 
```
parent_folder/
    agent_folder/
        __init__.py
        agent.py
        config.yaml
        .env
    deploy_agent_engine.ipynb
```

config.yaml (example): 
```yaml
agent_name: 'agent_name'
agent_display_name: 'Agent Name'
agent_description: 'Useful agent to help users'

deploy:
  dependencies: ['google-cloud-aiplatform[agent_engines]', 'google-adk', 'cloudpickle', 'pydantic']
```


### Setup and Config

In [ ]:
# Checking the google-adk and google-cloud-aiplatform versions
!pip freeze | grep google-adk
!pip freeze | grep google-cloud-aiplatform

In [ ]:
# Authentication on gcloud (if necessary)
!gcloud auth application-default login

In [ ]:
# Basic Libraries
import os 
import vertexai
import yaml

# AI Engine on Vertex AI 
from vertexai import agent_engines

# Library for AI Engine with ADK
from vertexai.preview import reasoning_engines

# Just to view JSON response formatted
import json
from IPython.display import display,Markdown,JSON

# To load envvars dict from .env file
from dotenv import dotenv_values

In [ ]:
# Load Agent Config
AGENT_DIR = "adk_vais_agent"

# Load environment variables from .env file from agent Directory 
from dotenv import load_dotenv
env_file = f'./{AGENT_DIR}/.env'
load_dotenv(env_file)

# Load config from agents (Params and dependencies for deploy)
with open(f'./{AGENT_DIR}/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# For Vertex AI SDK 
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")
BUCKET = os.environ.get("GOOGLE_CLOUD_BUCKET")

### Instantiate Agent from Directory

In [ ]:
# Importing Agent Module from AGENT_DIR folder
import importlib
agent_module = importlib.import_module(f"{AGENT_DIR.replace('/','.')}.agent")

# Instantiate the Assistant as an ADK App 
adk_agent = reasoning_engines.AdkApp(
    agent=agent_module.root_agent,
    enable_tracing=True
)

### Running Agent Local (Optional)

In [ ]:
# Run a simple query
for event in adk_agent.stream_query(
    user_id="user",
    message="Hi, how can you help me?",
):
    pass

# Formatted output
display(Markdown(f"```json\n{json.dumps(event, indent=2,ensure_ascii=False)}\n```"))

### Deploy on Agent Engine

In [ ]:
# Instantiate Vertex AI
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=BUCKET,
)

In [ ]:
## Retrieve all existent Agent Engine on your project
for agent in agent_engines.list():
    print(f"============================ \nAgent: {agent.display_name}\nResoruce Name: {agent.resource_name}\nCreated/updated at: {agent.update_time} \n\n" )

In [ ]:
# Read Requirements for Agent from config file
# Usually this ['google-cloud-aiplatform[agent_engines]', 'google-adk', 'cloudpickle'] plus the packages that agent needs
requirements = config['deploy']['dependencies']
requirements

In [ ]:
# Extra packages from agent folder (This is all .py files inside Agent Directory)
extra_packages = [AGENT_DIR]
extra_packages

In [ ]:
# Load Variables on env_vars dict to be used when creating the Agent
env_vars = dotenv_values(dotenv_path=env_file)

# Remove GCP variables (this variables already are defined in Agent Engine and are reserved)
keys_to_remove = [
    "GOOGLE_GENAI_USE_VERTEXAI",
    "GOOGLE_CLOUD_PROJECT",
    "GOOGLE_CLOUD_LOCATION",
    "GOOGLE_CLOUD_BUCKET"
]

for key in keys_to_remove:
    env_vars.pop(key, None)


In [ ]:
# Deploy the Agent on AI Engine (This takes a few minutes)
remote_agent = agent_engines.create(
    agent_engine = adk_agent,             # The Agent instantiated as ADK agent
    requirements=requirements,            # Requirements file
    extra_packages=extra_packages,        # Extra packages
    display_name=config['agent_display_name'],    # Display name  
    description=config['agent_description'],     # Description
    env_vars=env_vars                     # Env Vars dict
)

### Test Remote Agent on Agent Engine

In [ ]:
## Retrieve all existent Agent Engine resource.names (Agents)
# To confirm new agent was deployed
for agent in agent_engines.list():
    print(f"============================ \nAgent: {agent.display_name}\nResoruce Name: {agent.resource_name}\nCreated/updated at: {agent.update_time} \n\n" )

In [ ]:
# Confirm that "remote_agent" is pointing to your new agent
print(f"=================== Remote Agent ============================ \n\
 Name: {remote_agent.display_name}\n\
 Resoruce Name: {remote_agent.resource_name}\n\
 Created/updated at: {remote_agent.update_time} \n\n" )

In [ ]:
# Run a simple query
for remote_event in remote_agent.stream_query(
    user_id="user",
    message="Hi, what you can do form me?",
):
    display(JSON(remote_event,expanded=True)) 

In [ ]:
# Formatted final output
display(Markdown(f"```json\n{json.dumps(remote_event, indent=2,ensure_ascii=False)}\n```"))